In [1]:
cd /content/drive/MyDrive/Team_lemonmul/GMGM/Data

/content/drive/.shortcut-targets-by-id/1k2RzEvWtJ__WV4ThLWM5VyXk_FspqjfH/Team_lemonmul/GMGM/Data


In [4]:
### import 
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
import os
import sys
import urllib.request
import json, pprint
import pickle


### 상품 테이블 정의
goods_table = pd.read_csv('./goods_table_add.csv')


### 상품 단위용량 총용량 테이블 정의
goods_unit = pd.read_csv('./goods_unit_add.csv')


### 상품 이름-id 딕셔너리 정의
with open('./goodsName_goodsId_add.pickle', 'rb') as f:
    goods_id = pickle.load(f)


### 저장용 날짜 정의
today = datetime.datetime.now().strftime('%Y%m%d')


### api 키
client_id = "a8DDB5LJUVwh8_MWPtSj"
client_secret = "hKlWjoMufh"


### api 함수 정의
## 상품 가격 온라인 + cheap_url
def get_goods_online(goods_name):
  global cheap_url
  encText = urllib.parse.quote(goods_name)
  url = "https://openapi.naver.com/v1/search/shop.json?exclude=used:rental:cbshop&start=1&display=30&sort=sim&query=" + encText 
  request = urllib.request.Request(url)
  request.add_header("X-Naver-Client-Id",client_id)
  request.add_header("X-Naver-Client-Secret",client_secret)
  response = urllib.request.urlopen(request)
  rescode = response.getcode()

  if(rescode==200):
    response_body = response.read()
    res = json.loads(response_body.decode('utf-8'))
  else:
    print("Error Code:" + rescode)
    return
    
  online_info = dict()
  id = goods_id[goods_name]
  if not res['items']:
    cheap_url.append(None)
    return {'goods_id': id, 'price': None, 'research_date': datetime.datetime.now().strftime('%Y-%m-%d'), 'business': 'o'} 
  cheap = sorted(res['items'], key= lambda x: int(x['lprice']))[0]
  cheap_url.append(cheap.get('link'))
  online_info['goods_id'] = id
  if id < 20000:
    price = int(cheap.get('lprice')) * int(goods_unit.loc[id]['unit']) / int(goods_unit.loc[id]['총용량'])
    online_info['price'] = round(price, 2)
  else:
    online_info['price'] = cheap.get('lprice')
  online_info['research_date'] = datetime.datetime.now().strftime('%Y-%m-%d')
  online_info['business'] = 'o'

  return online_info

## 상품 가격 대형마트&SSM
def get_goods_info(mallName, goods_name):
  encText = urllib.parse.quote(mallName+goods_name)
  url = "https://openapi.naver.com/v1/search/shop.json?exclude=used:rental:cbshop&start=1&display=30&sort=sim&query=" + encText 
  request = urllib.request.Request(url)
  request.add_header("X-Naver-Client-Id",client_id)
  request.add_header("X-Naver-Client-Secret",client_secret)
  response = urllib.request.urlopen(request)
  rescode = response.getcode()

  if(rescode==200):
    response_body = response.read()
    res = json.loads(response_body.decode('utf-8'))
    # print(response_body.decode('utf-8'))
  else:
    print("Error Code:" + rescode)
    return
  goods_info = dict()
  id = goods_id[goods_name]
  goods_info['goods_id'] = id
  for i in range(res['display']):
    if mallName in res['items'][i].get('mallName').replace(' ',''):
      if id < 20000:
        price = int(res['items'][i].get('lprice')) * int(goods_unit.loc[id]['unit']) / int(goods_unit.loc[id]['총용량'])
        goods_info['price'] = round(price, 2)
      else:
        goods_info['price'] = res['items'][i].get('lprice')
      goods_info['research_date'] = datetime.datetime.now().strftime('%Y-%m-%d')
      break
  if mallName == 'GS프레시몰':
    goods_info['business'] = 's'
  else:
    goods_info['business'] = 'm'
  
  return goods_info


### 상품 가격 온라인 + cheap_url api 가져오기
cheap_url = []
o_data = []

for i in tqdm(range(len(goods_table))):
  goods_name = goods_table.iloc[i, 2]
  online_info = get_goods_online(goods_name)
  o_data.append(online_info)

## 상품 가격 온라인 저장
goods_price_o = pd.DataFrame(o_data)
goods_price_o.sort_values(by=['goods_id'], inplace=True)
goods_price_o.reset_index(drop=True, inplace=True)
goods_price_o.to_csv(f'goods_price_o_{today}.csv', index=False)
# DB 상품 가격 테이블에 추가

## 상품 테이블 cheap_url 갱신
goods_table['cheap_url'] = cheap_url
goods_table.to_csv(f'goods_table_{today}.csv', index=False)
# DB 상품 테이블 업데이트


### 상품 가격 대형마트 api 가져오기
m_data = []

for i in tqdm(range(len(goods_table))):
  goods_name = goods_table.iloc[i, 2]
  for market in ['이마트몰', 'Homeplus', '하나로마트']:
    m_data.append(get_goods_info(market, goods_name))

## 상품 가격 대형마트 저장
goods_price_m = pd.DataFrame(m_data)
goods_price_m['price'] = goods_price_m['price'].astype(float)
goods_price_m = goods_price_m.groupby(['goods_id', 'research_date', 'business'], as_index=False)['price'].mean().round(2)
goods_price_m = goods_price_m[['goods_id', 'research_date', 'price', 'business']]
goods_price_m.sort_values(by=['goods_id'], inplace=True)
goods_price_m.reset_index(drop=True, inplace=True)
goods_price_m.to_csv(f'goods_price_m_{today}.csv', index=False)
# DB 상품 가격 테이블에 추가


### 상품 가격 SSM api 가져오기
s_data = []

for i in tqdm(range(len(goods_table))):
  goods_name = goods_table.iloc[i, 2]
  for market in ['GS프레시몰']:
    s_data.append(get_goods_info(market, goods_name))

goods_price_s = pd.DataFrame(s_data)
goods_price_s = goods_price_s.groupby(['goods_id', 'research_date', 'business'], as_index=False)['price'].mean().round(2)
goods_price_s = goods_price_s[['goods_id', 'research_date', 'price', 'business']]
goods_price_s.sort_values(by=['goods_id'], inplace=True)
goods_price_s.reset_index(drop=True, inplace=True)
goods_price_s.to_csv(f'goods_price_s_{today}.csv', index=False)
# DB 상품 가격 테이블에 추가



100%|██████████| 527/527 [03:29<00:00,  2.52it/s]
